In [ ]:
import praw
import csv

# Reddit API credentials provided by PRAW
reddit = praw.Reddit(
    client_id="my client id",
    client_secret="my client secret",
    user_agent="my user agent",
)

# choosing which subreddit to scrape r/college, r/education, r/StudentLife
subreddit = reddit.subreddit('college')

#setting up container for data
posts_data = []

# fetching posts with search term 'group project'
post_batch = subreddit.search('group project', sort='top', time_filter='all', limit=None)

# code used to fix return error
for i, post in enumerate(post_batch):
    if i >= 1000:
        break
    
    # code used to fix empty/missing post_text
    post_text = post.selftext if post.selftext else ''
    
    # assigning data labels
    posts_data.append({
        'OP Username': post.author.name if post.author else '[unknown]',
        'Title': post.title,
        'Text': post_text,r
        'Upvotes': post.ups,
        'Downvotes': post.downs
    })

# Saving the data to a CSV file
csv_file_path = 'reddit_posts.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['OP Username', 'Title', 'Text', 'Upvotes', 'Downvotes']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for post_data in posts_data:
        writer.writerow(post_data)

print(f"Saved to {csv_file_path}")


In [ ]:
import openai
import pandas as pd

# OpenAI API key
openai.api_key = 'my api key'

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == "":
        return "null"  # returns "null" for blank or NaN rows
    
    # set prompt for AI analysis
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  
        messages=[
            {"role": "system", "content": "You are trained to analyze the sentiment of reddit posts. After reading the text, classify the sentiment as either 'positive' 'neutral' or 'negative' based on the posts view regarding group projects. Do not provide additional commentary or questions. Just assign a sentiment."},
            {"role": "user", "content": f"{text}"}
        ]
    )
    return response.choices[0].message['content'].strip()

# load the Reddit CSV file
df = pd.read_csv('/Users/Mojave760/Downloads/reddit_posts.csv')

# analyze sentiment for each post
df['Sentiment'] = df['text'].apply(analyze_sentiment)

# Saving to a new CSV file
filename = '/Users/Mojave760/Downloads/reddit_posts_with_sentiment.csv'
df.to_csv(filename, index=False)

print(f"Saved to {filename}")
